In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Calculating Theoretical Weights

In [2]:
weights = [1/.1428,1/.2081,1/.2038,1/.2050,1/.0617,1/.0594]

In [3]:
theo_weights = np.array([w/sum(weights) for w in weights])

# Model

In [4]:
risk = pd.read_csv('Processed')

In [5]:
risk.head()

,Date,S&P 500,Eurostoxx,NIKKEI,MSCI,Treasuries,Bonds
0,2004-12-30,NaN,NaN,NaN,177.658,NaN,NaN
1,2004-12-31,1799.55,4036.18,NaN,178.759,141.0920,141.8000
2,2005-01-02,1767.79,4121.14,14229.19,179.086,142.0369,142.9195
3,2005-01-03,1802.79,4220.82,14727.36,194.261,140.1816,141.1537
4,2005-01-04,1749.45,4198.11,14732.26,184.454,139.8953,139.9159


In [6]:
risk["Months"] = risk["Date"].apply(lambda x: x[:-3])

In [7]:
months = risk.groupby('Months').last().drop(columns = ['Date'])

In [8]:
months

,S&P 500,Eurostoxx,NIKKEI,MSCI,Treasuries,Bonds
Months,,,,,,
2004-12,1799.55,4036.18,NaN,178.759,141.0920,141.8000
2005-01,1755.68,4086.74,14233.18,179.213,142.0294,142.9191
2005-02,1792.63,4193.24,14677.44,194.849,140.2802,141.7296
2005-03,1760.89,4189.68,14663.55,181.972,139.5314,139.6564
2005-04,1727.49,4045.74,13834.11,177.087,142.8583,141.9775
...,...,...,...,...,...,...
2015-12,3821.60,6226.20,28507.92,341.623,246.5444,247.5175
2016-01,3631.96,5811.15,26240.18,319.461,254.2459,247.5039
2016-02,3627.06,5624.40,24021.78,318.938,257.9866,250.3032


In [9]:
def redistribute(row):
    '''Redistribute allocation of assests based on theoretical weights'''
    values = theo_weights * sum(row.values[0])
    df = pd.DataFrame(columns = ["S&P 500", "Eurostoxx", "NIKKEI", "MSCI", "Treasuries", "Bonds"])
    df.loc[0] = values
    df.index = row.index
    return df

In [10]:
def check_MA(test):
    '''For all equities and bonds, check if the 12 month moving average is less than the current asset value. 
       If the asset class is trading below 12M MA, then move to cash
       
       Output:
           Return list of booleans (True/False) determining if assets are in portfolio (True) or in cash (False)
    '''
    prev_year = pd.to_datetime(test.index) - pd.offsets.DateOffset(years= 1)
    prev_year = str(prev_year[0])[:-12]
    
    prev_month = pd.to_datetime(test.index) - pd.offsets.DateOffset(months= 1)
    prev_month = str(prev_month[0])[:-12]
    
    year_data = months.loc[prev_year: prev_month]
    
    cash_sp = cash_euro = cash_nikkei = cash_msci = cash_bonds = False
    
    if year_data["S&P 500"].mean() > test['S&P 500'][0]:
        cash_sp = True
    if year_data["Eurostoxx"].mean() > test['Eurostoxx'][0]:
        cash_euro = True
    if year_data["NIKKEI"].mean() > test['NIKKEI'][0]:
        cash_nikkei = True
    if year_data["MSCI"].mean() > test['MSCI'][0]:
        cash_msci = True
    if year_data["Bonds"].mean() > test['Bonds'][0]:
        cash_bonds = True
        
    return [not cash_sp, not cash_euro, not cash_nikkei, not cash_msci, True, not cash_bonds]

In [11]:
def update(portfolio, curr):
    '''
    Determine how much allocation changes by calculating change in asset value over the month.
    If the asset class is trading below 12M MA, set percent change of that asset to 0.
    
    Output:
        Return 1 + percentage change of asset value
    '''
    next_month = pd.to_datetime(curr.index) + pd.offsets.DateOffset(months= 1)
    next_month = str(next_month[0])[:-12]
    next_row = months.loc[[next_month]].fillna(0)
    
    perc_change = (next_row.values - curr.values)/curr.values
    
    perc_change = perc_change * np.array(check_MA(curr))
    
    result = portfolio * (1 + perc_change)
    result.reset_index(inplace=True)
    if "Months" in result.columns:
        result.drop(columns=['Months'], inplace = True)
    else:
        result.drop(columns=['index'], inplace = True)
    result.index = [next_month]    
    return result

In [12]:
from dateutil.relativedelta import relativedelta

In [13]:
def model(start, end):
    '''
    Inputs:
        Start and end dates
    
    Function:
        Model will take starting asset values, redistribute based on theoretical weights, 
        update according to investment rules, and then repeat until end month.
    
    Output:
        Return tuple containing
            1) Dataframe of monthly portfolio allocation
            2) Final total value of portfolio
            3) Total value of portfolio for each month
            4) CAGR
            5) Dataframe of monthly percentage allocation
        
    '''
    #Convert start date to Year-Month format
    start_ym = str(pd.to_datetime(start))[:-12]
    
    #Convert end date to Year-Month format. Add an extra month
    #We will stop our loop before we hit this month
    end_plus_one = pd.to_datetime(end) + pd.offsets.DateOffset(months= 1)
    stopping = str(end_plus_one)[:-12]
    
    next_month = start_ym
    
    #Don't get monthly return for first month
    prev_tot = False
    
    #curr is the row corresponding to the current month in the data matrix
    curr = months.loc[[start_ym]].fillna(0)
    
    #portfolio is the portfolio allocation per asset class in a given month
    portfolio = redistribute(curr)
    
    #Initial total value of portfolio
    initial_tot = sum(portfolio.values[0])
    
    #Total number of years
    num_years = relativedelta(pd.to_datetime(start), pd.to_datetime(end)).years + relativedelta(pd.to_datetime(start), pd.to_datetime(end)).months/12

    monthly_returns = []
    
    #Dataframe of monthly allocation in each asset class
    df = pd.DataFrame(columns = ["S&P 500", "Eurostoxx", "NIKKEI", "MSCI", "Treasuries", "Bonds"])
    
    #Total value of portfolio for each month
    totals = []
    
    #Dataframe of monthly percentage allocation
    percent_all = pd.DataFrame(columns = ["S&P 500", "Eurostoxx", "NIKKEI", "MSCI", "Treasuries", "Bonds"])
    percent_all.loc[0] = curr.values[0]/initial_tot
    i = 1
    
    #Repeat until end month.
    while next_month != stopping:
        #Calculate portfolio allocation and percentage allocation of current month 
        df = df.append(portfolio)
        portfolio = update(portfolio, curr)
        percent_all.loc[i] = portfolio.values[0]/sum(portfolio.values[0])
        i += 1
        portfolio = redistribute(portfolio)
        
        #Print monthly return
        curr_tot = sum(portfolio.values[0])
        totals.append(curr_tot)
        if prev_tot:  
            curr_return = (curr_tot - prev_tot)/prev_tot
            monthly_returns.append(round(curr_return * 100, 2))
        else:
            monthly_returns.append(0)
        prev_tot = curr_tot
                
        #Set current to next month
        next_month = pd.to_datetime(curr.index) + pd.offsets.DateOffset(months= 1)
        next_month = str(next_month[0])[:-12]
        curr = months.loc[[next_month]].fillna(0)
    
    cagr_out = round(cagr(curr_tot,initial_tot, num_years) * 100, 2)
    
    df['Monthly Returns (%)'] = monthly_returns
    
    percent_all.drop(i - 1, inplace=True)
    percent_all.index = df.index
    
    return (df, curr_tot, totals, cagr_out, percent_all)

In [14]:
def cagr(EV,BV, n):
    return (EV/BV)**(1/n) - 1

In [15]:
def volatility(output):
    output = output[0]
    # Equals the square root of the variance the portfolio
    # Equals the sum of sqrt of the variance of all asset classes each mulitiplied by the squared of their corresponding weight
    variance = (theo_weights[0]**2) * np.var(output['S&P 500']) + (theo_weights[1]**2) * np.var(output['Eurostoxx']) + (theo_weights[2]**2) * np.var(output['NIKKEI']) + (theo_weights[3]**2) * np.var(output['MSCI']) + (theo_weights[4]**2) *np.var(output['Treasuries'])+ (theo_weights[5]**2) * np.var(output['Bonds'])
    return np.sqrt(variance)

In [16]:
def sharpe(output):
    return output[1]/volatility(output)

In [17]:
def max_drawdown(output):
    return max(output[2]) - min(output[2])

# TEST PERIOD 1

In [18]:
test_period1 = model('1/31/2006', '2/29/2008')

### Monthly Allocation and Monthly Return

In [19]:
test_period1[0]

,S&P 500,Eurostoxx,NIKKEI,MSCI,Treasuries,Bonds,Monthly Returns (%)
2006-01,3679.826628,2525.128508,2578.406489,2563.313378,8516.681402,8846.451894,0.00
2006-02,3682.562300,2527.005750,2580.323339,2565.219007,8523.012909,8853.028560,0.07
2006-03,3684.981648,2528.665927,2582.018544,2566.904289,8528.612307,8858.844770,0.53
2006-04,3704.394049,2541.986882,2595.620560,2580.426684,8573.540846,8905.512967,-2.45
2006-05,3613.605584,2479.687061,2532.006268,2517.184768,8363.417786,8687.253828,0.17
2006-06,3619.882695,2483.994468,2536.404557,2521.557311,8377.945686,8702.344256,0.74
2006-07,3646.758816,2502.437092,2555.236305,2540.278824,8440.148443,8766.955537,2.40
2006-08,3734.182423,2562.427920,2616.492885,2601.176830,8642.483794,8977.125422,1.32
2006-09,3783.635617,2596.363124,2651.144093,2635.625200,8756.939482,9096.012897,1.64
2006-10,3845.571994,2638.864396,2694.542104,2678.769175,8900.286560,9244.910451,1.67


### Monthly Percentage Allocation

In [20]:
test_period1[4]

,S&P 500,Eurostoxx,NIKKEI,MSCI,Treasuries,Bonds
2006-01,0.067501,0.180435,0.732819,0.009275,0.005001,0.004969
2006-02,0.128426,0.089891,0.087373,0.089111,0.296259,0.308941
2006-03,0.129684,0.089823,0.094918,0.090011,0.292388,0.303177
2006-04,0.129213,0.087396,0.088535,0.095141,0.293196,0.306519
2006-05,0.127612,0.086329,0.084233,0.081936,0.304015,0.315875
2006-06,0.128124,0.088446,0.089949,0.088911,0.296971,0.307599
2006-07,0.128013,0.088410,0.088872,0.089876,0.298966,0.305863
2006-08,0.128151,0.088764,0.091610,0.089414,0.295137,0.306922
2006-09,0.129758,0.088880,0.088924,0.088850,0.296027,0.307562
2006-10,0.130218,0.088939,0.089852,0.092018,0.293573,0.305400


### CAGR (%):

In [21]:
test_period1[3]

-6.92

### Total Volatility:

In [22]:
volatility(test_period1)

235.94385353953928

### Sharpe Ratio:

In [23]:
sharpe(test_period1)

140.44761880877493

### Max Drawdown

In [24]:
max_drawdown(test_period1)

5328.157779782319

# TEST PERIOD 2

In [25]:
test_period2 = model('11/30/2007', '9/30/2012')

### Monthly Allocation and Monthly Return

In [26]:
test_period2[0]

,S&P 500,Eurostoxx,NIKKEI,MSCI,Treasuries,Bonds,Monthly Returns (%)
2007-11,3823.290479,2623.574629,2678.929737,2663.248197,8848.717672,9191.344787,0.00
2007-12,3819.380562,2620.891611,2676.190109,2660.524606,8839.668463,9181.945188,-1.56
2008-01,3759.943340,2580.105281,2634.543223,2619.121507,8702.105493,9039.055705,1.02
2008-02,3798.474933,2606.545990,2661.541808,2645.962051,8791.283962,9131.687213,-0.42
2008-03,3782.534555,2595.607566,2650.372593,2634.858217,8754.391158,9093.365900,-0.52
2008-04,3762.830529,2582.086494,2636.566239,2621.132680,8708.787674,9045.996625,-0.60
2008-05,3740.082463,2566.476577,2620.626966,2605.286711,8656.138990,8991.309355,-0.81
2008-06,3709.748802,2545.661359,2599.372566,2584.156726,8585.934018,8918.386009,-0.02
2008-07,3708.856335,2545.048941,2598.747226,2583.535047,8583.868472,8916.240483,0.49
2008-08,3726.849500,2557.396005,2611.354802,2596.068823,8625.512295,8959.496778,-2.52


### Monthly Percentage Allocation

In [27]:
test_period2[4]

,S&P 500,Eurostoxx,NIKKEI,MSCI,Treasuries,Bonds
2007-11,0.077855,0.217340,0.679393,0.014742,0.005475,0.005194
2007-12,0.127414,0.088145,0.089901,0.089690,0.296973,0.307876
2008-01,0.122390,0.077101,0.091229,0.079377,0.311718,0.318186
2008-02,0.126873,0.087061,0.088898,0.094901,0.297438,0.304828
2008-03,0.128713,0.088324,0.090188,0.084915,0.302089,0.305770
2008-04,0.128844,0.088414,0.090280,0.089751,0.291026,0.311685
2008-05,0.128953,0.088488,0.090355,0.091493,0.293200,0.307510
2008-06,0.129221,0.088673,0.090544,0.081034,0.302082,0.308446
2008-07,0.128204,0.087975,0.089831,0.089305,0.298360,0.306325
2008-08,0.127554,0.087529,0.089376,0.088852,0.300043,0.306646


### CAGR (%):

In [28]:
test_period2[3]

-4.31

### Total Volatility:

In [29]:
volatility(test_period2)

345.84148491596346

### Sharpe Ratio:

In [30]:
sharpe(test_period2)

106.721134367083

### Max Drawdown

In [31]:
max_drawdown(test_period2)

8667.469653729764

# TEST PERIOD 3

In [32]:
test_period3 = model('11/30/2012', '1/31/2015')

### Monthly Allocation and Monthly Return

In [33]:
test_period3[0]

,S&P 500,Eurostoxx,NIKKEI,MSCI,Treasuries,Bonds,Monthly Returns (%)
2012-11,2730.241475,1873.515053,1913.044566,1901.846257,6318.938130,6563.610819,0.00
2012-12,2769.971490,1900.778130,1940.882870,1929.521604,6410.890256,6659.123381,0.87
2013-01,2793.949805,1917.232254,1957.684161,1946.224547,6466.386258,6716.768217,0.75
2013-02,2815.021367,1931.691741,1972.448730,1960.902689,6515.154801,6767.425105,1.13
2013-03,2846.906938,1953.571892,1994.790534,1983.113711,6588.951552,6844.079305,2.83
2013-04,2927.592407,2008.938951,2051.325788,2039.318028,6775.691988,7038.050432,-1.54
2013-05,2882.398160,1977.926272,2019.658770,2007.836377,6671.093311,6929.401638,-3.12
2013-06,2792.581411,1916.293251,1956.725346,1945.271344,6463.219213,6713.478543,1.12
2013-07,2823.786259,1937.706284,1978.590175,1967.008184,6535.440482,6788.496259,-1.03
2013-08,2794.756401,1917.785748,1958.249333,1946.786410,6468.253065,6718.707309,2.20


### Monthly Percentage Allocation

In [34]:
test_period3[4]

,S&P 500,Eurostoxx,NIKKEI,MSCI,Treasuries,Bonds
2012-11,0.116511,0.212179,0.630183,0.018864,0.011233,0.011030
2012-12,0.127486,0.088803,0.097555,0.092306,0.289915,0.303935
2013-01,0.133655,0.089639,0.095413,0.089738,0.289989,0.301566
2013-02,0.128941,0.085070,0.092572,0.087502,0.297610,0.308305
2013-03,0.131491,0.086716,0.095929,0.086763,0.294581,0.304520
2013-04,0.127042,0.088990,0.097640,0.087477,0.292504,0.306346
2013-05,0.133228,0.092246,0.090649,0.088357,0.292441,0.303078
2013-06,0.130519,0.085481,0.092165,0.086289,0.298259,0.307287
2013-07,0.133207,0.092650,0.088765,0.088297,0.292353,0.304728
2013-08,0.125754,0.087390,0.088945,0.090211,0.296366,0.311334


### CAGR (%):

In [35]:
test_period3[3]

-7.38

### Total Volatility:

In [36]:
volatility(test_period3)

123.11324910169883

### Sharpe Ratio:

In [37]:
sharpe(test_period3)

204.28336809170992

### Max Drawdown

In [38]:
max_drawdown(test_period3)

3538.821452257187

# TEST PERIOD 4

In [39]:
test_period4 = model('2/29/2008', '12/31/2014')

### Monthly Allocation and Monthly Return

In [40]:
test_period4[0]

,S&P 500,Eurostoxx,NIKKEI,MSCI,Treasuries,Bonds,Monthly Returns (%)
2008-02,3325.513738,2281.995972,2330.144072,2316.504204,7696.650921,7994.669392,0.00
2008-03,3311.558152,2272.419530,2320.365575,2306.782947,7664.351768,7961.119598,-0.52
2008-04,3294.307542,2260.582013,2308.278297,2294.766424,7624.426531,7919.648434,-0.60
2008-05,3274.391916,2246.915740,2294.323678,2280.893491,7578.333316,7871.770464,-0.81
2008-06,3247.835204,2228.692297,2275.715736,2262.394473,7516.869807,7807.927055,-0.02
...,...,...,...,...,...,...,...
2014-08,4724.993738,3242.331118,3310.741441,3291.361491,10935.641908,11359.075854,-1.05
2014-09,4675.253062,3208.198641,3275.888799,3256.712864,10820.520863,11239.497260,1.09
2014-10,4726.368694,3243.274625,3311.704855,3292.319266,10938.824140,11362.381304,1.95
2014-11,4818.567930,3306.542530,3376.307657,3356.543904,11152.212324,11584.031993,-0.73


### Monthly Percentage Allocation

In [41]:
test_period4[4]

,S&P 500,Eurostoxx,NIKKEI,MSCI,Treasuries,Bonds
2008-02,0.080842,0.212060,0.678460,0.015985,0.006597,0.006056
2008-03,0.128713,0.088324,0.090188,0.084915,0.302089,0.305770
2008-04,0.128844,0.088414,0.090280,0.089751,0.291026,0.311685
2008-05,0.128953,0.088488,0.090355,0.091493,0.293200,0.307510
2008-06,0.129221,0.088673,0.090544,0.081034,0.302082,0.308446
...,...,...,...,...,...,...
2014-08,0.130809,0.087914,0.087073,0.089589,0.296343,0.308273
2014-09,0.127720,0.090493,0.095723,0.083545,0.296660,0.305858
2014-10,0.129884,0.084054,0.090164,0.089359,0.298096,0.308442
2014-11,0.129102,0.090191,0.093713,0.086649,0.295065,0.305280


### CAGR (%):

In [42]:
test_period4[3]

-5.53

### Total Volatility:

In [43]:
volatility(test_period4)

491.06695711222943

### Sharpe Ratio:

In [44]:
sharpe(test_period4)

77.91125604530473

### Max Drawdown

In [45]:
max_drawdown(test_period4)

13534.917344856643

# TEST PERIOD 5

In [46]:
test_period5 = model('12/31/2014', '12/31/2015')

### Monthly Allocation and Monthly Return

In [47]:
test_period5[0]

,S&P 500,Eurostoxx,NIKKEI,MSCI,Treasuries,Bonds,Monthly Returns (%)
2014-12,4639.472643,3183.645812,3250.817927,3231.788749,10737.709781,11153.479688,0.00
2015-01,4756.432194,3263.904456,3332.769957,3313.261060,11008.403846,11434.655174,0.93
2015-02,4800.748630,3294.314774,3363.821906,3344.131241,11110.970898,11541.193677,0.69
2015-03,4834.043523,3317.162014,3387.151202,3367.323976,11188.029417,11621.235943,-0.43
2015-04,4813.047906,3302.754642,3372.439848,3352.698736,11139.436645,11570.761633,0.03
2015-05,4814.431763,3303.704257,3373.409499,3353.662711,11142.639478,11574.088482,-1.82
2015-06,4726.667782,3243.479862,3311.914422,3292.527606,10939.516358,11363.100324,1.24
2015-07,4785.091755,3283.570892,3352.851338,3333.224891,11074.734241,11503.553917,-2.21
2015-08,4679.466101,3211.089665,3278.840820,3259.647606,10830.271624,11249.625576,-0.25
2015-09,4667.747516,3203.048272,3270.629761,3251.484611,10803.149844,11221.453625,-0.17


### Monthly Percentage Allocation

In [48]:
test_period5[4]

,S&P 500,Eurostoxx,NIKKEI,MSCI,Treasuries,Bonds
2014-12,0.104137,0.161639,0.709538,0.011093,0.006704,0.006889
2015-01,0.121268,0.091531,0.088730,0.087088,0.300669,0.310715
2015-02,0.134289,0.093595,0.094689,0.088459,0.287217,0.301751
2015-03,0.125277,0.089811,0.091669,0.088669,0.297296,0.307277
2015-04,0.129967,0.086691,0.091674,0.089673,0.296122,0.305874
2015-05,0.129784,0.087825,0.094582,0.085683,0.296123,0.306004
2015-06,0.128026,0.086066,0.090215,0.090941,0.297283,0.307469
2015-07,0.129261,0.091411,0.090250,0.088193,0.296513,0.304371
2015-08,0.123159,0.081732,0.084321,0.091299,0.304402,0.315089
2015-09,0.128495,0.088174,0.083428,0.089508,0.301488,0.308907


### CAGR (%):

In [49]:
test_period5[3]

0.75

### Total Volatility:

In [50]:
volatility(test_period5)

73.33521361711539

### Sharpe Ratio:

In [51]:
sharpe(test_period5)

489.9001652507525

### Max Drawdown

In [52]:
max_drawdown(test_period5)

1788.0128039613337